In [60]:
v0=[0.301927,0.918442,0.781294,0.110588,0.997035,0.8792,0.283908,0.836897,0.10642,0.999105,0.665685,0.650125,0.0904407,0.897033,0.0289995,0.240828,0.143022,0.776768,0.198204,0.910638,0.656269,0.0361627,0.00542983,0.0516579,0.605925,0.801482,0.238553,0.849409,0.0572319,0.800964,0.927795,0.772108,0.698121,0.83798,0.0401513,0.201782,0.124924,0.504531,0.745188,0.630012,0.851131,0.155213,0.734621,0.193041,0.270759,0.709905,0.980205,0.611544,0.0545003,0.616309,0.0423506,0.884146,0.709578,0.173128,0.091721,0.183533,0.980027,0.458561,0.784081,0.636408,0.572413,0.14513,0.946024,0.301343,0.578017,0.699776,0.649233,0.940594,0.148439,0.508353,0.404034,0.474169,0.119218,0.134095,0.278076,0.304705,0.427903,0.610988,0.634629,0.411811,0.408783,0.217629,0.588306,0.317041,0.0360598,0.4184,0.474133,0.225593,0.572458,0.565772,0.702002,0.647949,0.652433,0.316214,0.787432,0.549144,0.431418,0.626013,0.360657,0.512739];
print(len(v0))

from numpy.linalg import norm
print(norm(v0))
print(v0 / norm(v0))

100
5.731440656391118


In [1]:
def _set_diag(laplacian, value, norm_laplacian):
    """Set the diagonal of the laplacian matrix and convert it to a
    sparse format well suited for eigenvalue decomposition.

    Parameters
    ----------
    laplacian : {ndarray, sparse matrix}
        The graph laplacian.

    value : float
        The value of the diagonal.

    norm_laplacian : bool
        Whether the value of the diagonal should be changed or not.

    Returns
    -------
    laplacian : {array, sparse matrix}
        An array of matrix in a form that is well suited to fast
        eigenvalue decomposition, depending on the band width of the
        matrix.
    """
    n_nodes = laplacian.shape[0]
    # We need all entries in the diagonal to values
    if not sparse.issparse(laplacian):
        if norm_laplacian:
            laplacian.flat[:: n_nodes + 1] = value
    else:
        laplacian = laplacian.tocoo()
        if norm_laplacian:
            diag_idx = laplacian.row == laplacian.col
            laplacian.data[diag_idx] = value
        # If the matrix has a small number of diagonals (as in the
        # case of structured matrices coming from images), the
        # dia format might be best suited for matvec products:
        n_diags = np.unique(laplacian.row - laplacian.col).size
        if n_diags <= 7:
            # 3 or less outer diagonals on each side
            laplacian = laplacian.todia()
        else:
            # csr has the fastest matvec and is thus best suited to
            # arpack
            laplacian = laplacian.tocsr()
    return laplacian

In [4]:
%reload_ext autoreload
%autoreload 2
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
from cuml.manifold.lanczos import eig_lanczos
from scipy.sparse.linalg import lobpcg
from scipy.sparse import random as sparse_random
from scipy.sparse import rand as sparse_rand
from scipy.sparse.linalg import eigs, eigsh
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import kneighbors_graph
from scipy.sparse.csgraph import laplacian as csgraph_laplacian
from scipy import sparse
import time
from sklearn.manifold import SpectralEmbedding
from cuml.manifold import SpectralEmbedding as cuSpectralEmbedding
from scipy.sparse.linalg import ArpackError
from cupyx.scipy.sparse.linalg import eigsh as cueigsh
from cupyx.scipy.sparse import csr_matrix as cucsr_matrix

np.set_printoptions(precision=4, suppress=True, threshold=50)


n_samples = 1000
n_features = 1000
n_components = 100
n_neighbors = 10
sparsity = 0.02
start = time.time()
rng = np.random.default_rng(42)
X = sparse_random(n_samples, n_features, density=sparsity, random_state=rng, dtype=np.float32)
# print(time.time() - start)
# start = time.time()
X = X + X.T
print(time.time() - start)
print(X.shape, X.dtype, type(X))
print("nnz", X.nnz)
cuX = cucsr_matrix(X)
print(cuX.dtype)


# X = X.astype(np.float32)
# X = X + X.T
# X = kneighbors_graph(
#     X, n_neighbors, include_self=True, mode="distance",
# )
# X = X + X.T
# matrix1 = X

# laplacian, dd = csgraph_laplacian(
#     X, normed=False, return_diag=True
# )
# laplacian = _set_diag(laplacian, 1, False)
# # X = laplacian + laplacian.T
# X = laplacian

# print(X)

times = {"Lobpcg Sklearn Time (s)": [], "Eigsh Sklearn Time (s)": [], "Raft Time (s)": [], "Cupy eigsh Sklearn Time (s)": []}
values = {"Lobpcg Sklearn Time (s)": [], "Eigsh Sklearn Time (s)": [], "Raft Time (s)": [], "Cupy eigsh Sklearn Time (s)": []}

# import rmm
# pool = rmm.mr.PoolMemoryResource(
#     rmm.mr.CudaMemoryResource(),
#     initial_pool_size=2**32,
#     maximum_pool_size=4*2**32,
# )
# rmm.mr.set_current_device_resource(pool)

for i in range(5):
    r = np.random.RandomState(42)
    X0 = r.standard_normal(size=(n_samples, n_components + 1))
    
    start = time.time()
    eigenvalues, eigenvectors = lobpcg((X), X=X0, tol=1e-9, largest=False, maxiter=10000)
    times["Lobpcg Sklearn Time (s)"].append(time.time() - start)
    values["Lobpcg Sklearn Time (s)"].append(eigenvalues)
    
    try:
        start = time.time()
        eigenvalues, eigenvectors = eigsh((X), k=n_components, which="SA", maxiter=10000, tol=1e-9)
        times["Eigsh Sklearn Time (s)"].append(time.time() - start)
        values["Eigsh Sklearn Time (s)"].append(eigenvalues)
    except ArpackError as e:
        print(e)
    
    start = time.time()
    eigenvalues, eigenvectors, eig_iters = eig_lanczos((X), n_components, 42, dtype=np.float32, maxiter=1000, tol=1e-9, conv_n_iters=4, conv_eps=0.001, restartiter=50, handle=None)
    times["Raft Time (s)"].append(time.time() - start)
    values["Raft Time (s)"].append(eigenvalues)
    
    
    start = time.time()
    eigenvalues, eigenvectors = cueigsh(cuX, k=n_components, which="SA", maxiter=10000, tol=1e-9)
    times["Cupy eigsh Sklearn Time (s)"].append(time.time() - start)
    values["Cupy eigsh Sklearn Time (s)"].append(eigenvalues)
# dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
# print(dot1_double, dot2_double, dot3_double)


for (k1, v1), (k2, v2) in zip(times.items(), values.items()):
    print(k1, sum(v1)/len(v1))
    print(v2[0])


# start = time.time()
# s = SpectralEmbedding(n_components=n_components, n_neighbors=n_neighbors, random_state=42, eigen_solver="lobpcg")
# s.affinity_matrix_ = matrix1
# out = s.fit_transform(matrix1)
# print("SpectralEmbedding Time (s)", time.time() - start)

# matrix1 = matrix1.tocoo()
# start = time.time()
# s = cuSpectralEmbedding(n_components=n_components, n_neighbors=n_neighbors, random_state=42, rows=matrix1.row, cols=matrix1.col, vals=matrix1.data, nnz=len(matrix1.row))
# out = s.fit_transform(matrix1.toarray())
# print("Raft SpectralEmbedding Time (s)", time.time() - start)


0.008818387985229492
(1000, 1000) float32 <class 'scipy.sparse._csr.csr_matrix'>
nnz 39639
float32


MemoryError: std::bad_alloc: out_of_memory: CUDA error at: /home/coder/.conda/envs/rapids/include/rmm/mr/device/cuda_memory_resource.hpp:60: cudaErrorMemoryAllocation out of memory

In [45]:
start = time.time()
eigenvalues, eigenvectors, eig_iters = eig_lanczos(X, 3, 42, dtype=np.float32, maxiter=1000, tol=1e-9, conv_n_iters=4, conv_eps=0.01, restartiter=400, handle=None)
print("Raft Time (s)", time.time() - start)
print(eigenvalues, eig_iters)
print(eigenvectors)
dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
print(np.array([dot1_double, dot2_double, dot3_double]))

seed 42
Raft Time (s) 5.14221978187561
[-51.2329 -51.1512 -51.0847] 124
[[ 0.0031 -0.0001 -0.0014]
 [-0.0044 -0.001  -0.0038]
 [ 0.0034  0.0015  0.0013]
 ...
 [ 0.002  -0.0009 -0.0019]
 [ 0.0034 -0.0021 -0.0018]
 [ 0.0007  0.0026 -0.0028]]
[ 0. -0. -0.]


In [42]:
from scipy.sparse.linalg import eigs, eigsh
start = time.time()
eigenvalues, eigenvectors = eigsh(X, k=3, which="SA", maxiter=100, tol=0.01)
print("Eigsh Sklearn Time (s)", time.time() - start)
print(eigenvalues)
print(eigenvectors)
dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
print(np.array([dot1_double, dot2_double, dot3_double]))

Eigsh Sklearn Time (s) 120.89733219146729
[-50.6569 -49.9981 -49.983 ]
[[ 0.0011 -0.0043  0.005 ]
 [-0.0024  0.0084 -0.0036]
 [-0.0012 -0.0015 -0.0039]
 ...
 [ 0.0056 -0.0055  0.0004]
 [-0.0037  0.0026 -0.0027]
 [-0.003  -0.0043  0.0014]]
[ 0. -0. -0.]


In [44]:
r = np.random.RandomState(42)
X0 = r.standard_normal(size=(n_samples, n_components + 1))

start = time.time()
eigenvalues, eigenvectors = lobpcg(X, X=X0, tol=0.1, largest=False, maxiter=100)
print("Lobpcg Sklearn Time (s)", time.time() - start)
print(eigenvalues)
print(eigenvectors)

KeyboardInterrupt: 

In [37]:
from cupyx.scipy.sparse.linalg import eigsh as cueigsh
cuX = cucsr_matrix(X)

In [39]:
from cupyx.scipy.sparse.linalg import eigsh as cueigsh
from cupyx.scipy.sparse import csr_matrix as cucsr_matrix

start = time.time()
eigenvalues, eigenvectors = cueigsh(cuX, k=3, which="SA", maxiter=100, tol=0.001)
print("Cupy eigsh Sklearn Time (s)", time.time() - start)
print(eigenvalues)
print(eigenvectors)
dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
print(dot1_double, dot2_double, dot3_double)



Eigsh Sklearn Time (s) 1.5047388076782227
[-51.0874 -51.0472 -51.017 ]
[[-0.0028 -0.0026 -0.0045]
 [ 0.0028  0.0016  0.0035]
 [ 0.0029  0.0005 -0.0005]
 ...
 [ 0.0043  0.0022  0.0021]
 [-0.003  -0.0036 -0.003 ]
 [ 0.003  -0.0088 -0.006 ]]
-0.0029701418 -0.0014672056 -0.009164453


In [51]:
import numpy as np
from scipy.sparse import diags

def generate_tridiagonal_matrix(n, mean=0, std=1):
    """
    Generate a large sparse tridiagonal matrix with normally distributed diagonal entries.
    
    Parameters:
    n (int): Size of the matrix (n x n).
    mean (float): Mean of the normal distribution for diagonal entries.
    std (float): Standard deviation of the normal distribution for diagonal entries.
    
    Returns:
    scipy.sparse.csr_matrix: Sparse tridiagonal matrix in CSR format.
    """
    # Generate the diagonal entries
    main_diag = np.random.normal(loc=mean, scale=std, size=n)
    
    # Generate the sub-diagonal and super-diagonal entries
    off_diag = np.random.normal(loc=mean, scale=std, size=n-1)
    
    # Create the sparse tridiagonal matrix
    diagonals = [off_diag, main_diag, off_diag]
    offsets = [-1, 0, 1]
    sparse_matrix = diags(diagonals, offsets, shape=(n, n), format='csr')
    
    return sparse_matrix

n = 100
tridiagonal_matrix = generate_tridiagonal_matrix(n)
print(tridiagonal_matrix)
# Convert to dense format for verification (only for small matrices)
dense_matrix = tridiagonal_matrix.toarray()

# Check if the matrix is symmetric
is_symmetric = np.allclose(dense_matrix, dense_matrix.T)

print(f"The matrix is symmetric: {is_symmetric}")

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 299998 stored elements and shape (100000, 100000)>
  Coords	Values
  (0, 0)	-1.161800846694018
  (0, 1)	-0.042400693787638104
  (1, 0)	-0.042400693787638104
  (1, 1)	-0.03908537005609671
  (1, 2)	0.27308032808376276
  (2, 1)	0.27308032808376276
  (2, 2)	-0.7958440161423127
  (2, 3)	1.0022925538478584
  (3, 2)	1.0022925538478584
  (3, 3)	-0.5591235294797839
  (3, 4)	-0.9558101747874178
  (4, 3)	-0.9558101747874178
  (4, 4)	0.5813940005459223
  (4, 5)	-2.2742399709316223
  (5, 4)	-2.2742399709316223
  (5, 5)	0.8027574089743723
  (5, 6)	-1.5570759749000602
  (6, 5)	-1.5570759749000602
  (6, 6)	0.47029335852435933
  (6, 7)	0.669775608341218
  (7, 6)	0.669775608341218
  (7, 7)	-0.8588042441818531
  (7, 8)	-0.46632773895408974
  (8, 7)	-0.46632773895408974
  (8, 8)	-0.2765081313798636
  :	:
  (99991, 99991)	-0.9434375655915473
  (99991, 99992)	0.1497153026997709
  (99992, 99991)	0.1497153026997709
  (99992, 99992)	1.87451589632710

KeyboardInterrupt: 

In [54]:
from scipy.sparse.linalg import eigs, eigsh
start = time.time()
eigenvalues, eigenvectors = eigsh(tridiagonal_matrix, k=3, which="SA", maxiter=100, tol=0.001)
print("Eigsh Sklearn Time (s)", time.time() - start)
print(eigenvalues)
print(eigenvectors)
dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
print(np.array([dot1_double, dot2_double, dot3_double]))

Eigsh Sklearn Time (s) 0.271151065826416
[-5.5744 -5.3756 -5.3042]
[[-0.  0. -0.]
 [-0.  0. -0.]
 [-0.  0. -0.]
 ...
 [ 0. -0.  0.]
 [-0.  0. -0.]
 [-0.  0. -0.]]
[-0. -0. -0.]


In [62]:
start = time.time()
eigenvalues, eigenvectors, eig_iters = eig_lanczos(tridiagonal_matrix, 3, 42, dtype=np.float32, maxiter=1000, tol=1e-9, conv_n_iters=4, conv_eps=0.001, restartiter=400, handle=None)
print("Raft Time (s)", time.time() - start)
print(eigenvalues, eig_iters)
print(eigenvectors)
dot1_double, dot2_double, dot3_double = np.dot(eigenvectors[:, 0], eigenvectors[:, 1]), np.dot(eigenvectors[:, 0], eigenvectors[:, 2]), np.dot(eigenvectors[:, 1], eigenvectors[:, 2])
print(np.array([dot1_double, dot2_double, dot3_double]))
print(np.count_nonzero(eigenvectors[:, 1]), eigenvectors[:, 1].shape)
print(eigenvectors[:, 1][0], eigenvectors[:, 1].shape)

seed 42
Raft Time (s) 0.8112988471984863
[-5.5744 -5.3756 -5.304 ] 75
[[ 0. -0.  0.]
 [ 0. -0.  0.]
 [-0. -0. -0.]
 ...
 [-0.  0. -0.]
 [-0.  0. -0.]
 [-0.  0. -0.]]
[ 0. -0. -0.]
100000 (100000,)
-4.595364e-10 (100000,)
